# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
from tqdm import tqdm
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy import create_engine, MetaData, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from tqdm import tqdm
import pandas as pd
import time

# Define maximum allowed connections
MAX_CONNECTIONS = 10

# Track the number of active connections
active_connections = 0

def execute_query(query, params=None):
    global active_connections

    # Ensure active connections do not exceed the maximum limit
    if active_connections >= MAX_CONNECTIONS:
        print("Maximum connection limit reached. Cannot execute query.")
        return None

    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Increment active connections count
        active_connections += 1

        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            # session.commit()
            return rows_affected
    except SQLAlchemyError as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
        return None
    finally:
        # Decrement active connections count
        active_connections -= 1

        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def remove_mentions(tweet):
    # Define regular expression pattern to match mentions
    pattern = r'@\w+'
    
    # Replace mentions with an empty string
    cleaned_tweet = re.sub(pattern, '', tweet)
    
    return cleaned_tweet

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 

    # we calculate sentiment for 'title' and 'description' column

    # remove first
    source_ds['title'] = source_ds['title'].apply(clean_up_tag)
    source_ds['description'] = source_ds['description'].apply(clean_up_tag)


    # skip stemming
    source_ds['stemmed'] = source_ds['title']
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    # tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        res = pipe(list_text[i])
        #print(res)
        new_list.append(res)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# Set up logging to a file
import logging
logging.basicConfig(filename='error.log', level=logging.ERROR)

def calculate_sentiment(row, col_description):
    try:
        # cleansing
        strSource = row[col_description]

        # cleansing
        strSource = clean_up_tag(strSource)
        strSource = casefolding(strSource)
        strSource = text_preproc(strSource)
        strSource = remove_mentions(strSource)

        # tokenized
        sToken = nltk.word_tokenize(strSource)

        # remove stopwords
        sw = stopwords.words('indonesian')

        filtered_words = [word for word in sToken if word.lower() not in sw]
        filtered_text = ' '.join(filtered_words)

        # Notes on return sentiment values
        # 1 >> positif >> stay
        # 2 >> negatif >> convert to -1
        # 0 >> netral >> stay
        result = pipe(strSource)

        if result[0]['label'] == 'Negative':
            sentimentClass = -1
        elif result[0]['label'] == 'Positive':
            sentimentClass = 1
        else:
            sentimentClass = 0

        score = result[0]['score']

        # put to result    
        return pd.Series({'sentiment_category': sentimentClass, 'score': score})

    except Exception as e:
        
        # Log the exception to the error.log file
        logging.error(f"An error occurred: {e}")
        # Return default values or None for the columns
        return pd.Series({'sentiment_category': None, 'score': None})

    

In [ ]:
#
# Calculate sentiment from table twitter_tweets and record to rdbms
# 
select_query = """
SELECT t.id, t.tweet
FROM twitter_tweets t
LEFT JOIN twitter_tweets_result_sentiment r ON t.id = r.twitter_tweets_id
WHERE r.twitter_tweets_id IS NULL
ORDER BY t.id desc limit 10000
"""

source_ds = execute_query(select_query)

if len(source_ds) != 0:
    for i in tqdm(range(len(source_ds))):
        row = source_ds.iloc[i]
        id = row['id']
        out_row = calculate_sentiment(row, 'tweet')
        
        ssql = """
INSERT into twitter_tweets_result_sentiment (sentiment_category, score, twitter_tweets_id)
VALUES (%s, %s, %s);
""" %(out_row['sentiment_category'],out_row['score'],row['id'])
        execute_query(ssql)




 35%|███████████████████████████████████████████████                                                                                       | 3509/10000 [07:54<14:45,  7.33it/s]

Error executing query: (psycopg2.OperationalError) connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already
connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already

(Background on this error at: https://sqlalche.me/e/20/e3q8)


 35%|███████████████████████████████████████████████                                                                                       | 3511/10000 [07:54<13:51,  7.81it/s]

Error executing query: (psycopg2.OperationalError) connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already
connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already

(Background on this error at: https://sqlalche.me/e/20/e3q8)
Error executing query: (psycopg2.OperationalError) connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already
connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already

(Background on this error at: https://sqlalche.me/e/20/e3q8)


 35%|███████████████████████████████████████████████▌                                                                                      | 3545/10000 [07:59<14:22,  7.48it/s]

...

 48%|████████████████████████████████████████████████████████████████▌                                                                     | 4816/10000 [10:33<10:03,  8.59it/s]

Error executing query: (psycopg2.OperationalError) connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already
connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already

(Background on this error at: https://sqlalche.me/e/20/e3q8)


 48%|████████████████████████████████████████████████████████████████▌                                                                     | 4818/10000 [10:33<10:28,  8.25it/s]

Error executing query: (psycopg2.OperationalError) connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already
connection to server at "98.98.117.105", port 5432 failed: FATAL:  sorry, too many clients already

(Background on this error at: https://sqlalche.me/e/20/e3q8)


 73%|█████████████████████████████████████████████████████████████████████████████████████████████████▉                                    | 7313/10000 [16:00<05:52,  7.62it/s]


In [ ]:
# 
# calculate sentiment result for google_result
# 

def record_google_sentiment(row):
    ssql = """
    INSERT INTO public.google_result_sentiment 
    (id, sentiment_category, score, google_result_id) 
    VALUES(nextval('google_result_sentiment_id_seq'::regclass), %s, %s, %s);
    """ %(row['sentiment_category'],row['score'],row['google_result_id'])
    #
    # print(ssql)
    execute_query(ssql)

select_query = "select * \
from google_result gr \
where google_result_id not in (select google_result_id from google_result_sentiment) \
order by google_result_id desc limit 100"

source_ds2 = execute_query(select_query)

if len(source_ds2) != 0:
    source_ds2[['sentiment_category','score']] = source_ds2.apply(calculate_sentiment, args=('title',), axis=1)
    source_ds2.apply(record_google_sentiment, axis=1)
